In [23]:
from tespy.components import PolynomialCompressor, Source, Sink
from tespy.connections import Connection
from tespy.networks import Network


class PolynomialCompressorWithCooling(PolynomialCompressor):

    @staticmethod
    def inlets():
        return ['in1', 'in2']

    @staticmethod
    def outlets():
        return ['out1', 'out2']


nw = Network()
nw.units.set_defaults(
    temperature="°C",
    pressure="bar"
)


gas_inlet = Source("gas inlet")
gas_outlet = Sink("gas outlet")

water_inlet = Source("water cold")
water_outlet = Sink("water hot")

compressor = PolynomialCompressorWithCooling("compressor")

c1 = Connection(gas_inlet, "out1", compressor, "in1", label="c1")
c2 = Connection(compressor, "out1", gas_outlet, "in1", label="c2")

b1 = Connection(water_inlet, "out1", compressor, "in2", label="b1")
b2 = Connection(compressor, "out2", water_outlet, "in1", label="b2")


nw.add_conns(c1, c2, b1, b2)

In [24]:
c1.set_attr(fluid={"R290": 1}, m=1, T_dew=10, td_dew=10)
c2.set_attr(T_dew=60, td_dew=50)

b1.set_attr(fluid={"water": 1}, m=0.5, T=15, p=1)
b2.set_attr(T=25, p=1)

compressor.set_attr(dissipation_ratio=0.1)

nw.solve("design")


 iter  | residual   | progress   | massflow   | pressure   | enthalpy   | fluid      | component  
-------+------------+------------+------------+------------+------------+------------+------------
 1     | 0.00e+00   | 100 %      | 0.00e+00   | 0.00e+00   | 0.00e+00   | 0.00e+00   | 0.00e+00   
 2     | 0.00e+00   | 100 %      | 0.00e+00   | 0.00e+00   | 0.00e+00   | 0.00e+00   | 0.00e+00   
 3     | 0.00e+00   | 100 %      | 0.00e+00   | 0.00e+00   | 0.00e+00   | 0.00e+00   | 0.00e+00   
 4     | 0.00e+00   | 100 %      | 0.00e+00   | 0.00e+00   | 0.00e+00   | 0.00e+00   | 0.00e+00   
Total iterations: 4, Calculation time: 0.00 s, Iterations per second: 1805.94


- the base component class has pairwise mass flow and fluid composition balance:
  - in1 matches with out1
  - in2 matches with out2
  - ...
- this is automatically expanded with every new pair of ports
- if you want to add ports with other names or non-paired ports, this may break
- in this case we can verify, if things went the right way by checking mass flow
  and fluid composition in results

In [25]:
nw.print_results()


##### RESULTS (PolynomialCompressorWithCooling) #####
+------------+----------+-----------+---------------------+
|            |       pr |        dp |   dissipation_ratio |
|------------+----------+-----------+---------------------|
| compressor | 3.33e+00 | -1.48e+01 |            1.00e-01 |
+------------+----------+-----------+---------------------+
##### RESULTS (Connection) #####
+----+-----------+-----------+-----------+-----------+------------+----------------------+
|    |         m |         p |         h |         T |          x | phase                |
|----+-----------+-----------+-----------+-----------+------------+----------------------|
| b1 | 5.000e-01 | 1.000e+00 | 6.308e+04 | 1.500e+01 |  0.000e+00 | l                    |
| b2 | 5.000e-01 | 1.000e+00 | 1.049e+05 | 2.500e+01 |  0.000e+00 | l                    |
| c1 | 1.000e+00 | 6.366e+00 | 6.040e+05 | 2.000e+01 |  1.000e+00 | g                    |
| c2 | 1.000e+00 | 2.117e+01 | 7.502e+05 | 1.100e+02 | -1.000e+00 

In [26]:
b1.fluid.val, b2.fluid.val

({'water': 1.0}, {'water': 1.0})

In [27]:
from tespy.tools.data_containers import ComponentMandatoryConstraints as dc_cmc


class PolynomialCompressorWithCooling(PolynomialCompressor):

    def _preprocess(self, row_idx):
        if not self.dissipation_ratio.is_set:
            self.dissipation_ratio.is_set = True
            self.dissipation_ratio.val = 0
            self.dissipation_ratio.val_SI = 0
        return super()._preprocess(row_idx)

    @staticmethod
    def inlets():
        return ['in1', 'in2']

    @staticmethod
    def outlets():
        return ['out1', 'out2']

    def get_mandatory_constraints(self) -> dict:
        constraints = super().get_mandatory_constraints()
        # this is a dictionary
        constraints["cooling_energy_balance_constraints"] = dc_cmc(
            func=self.cooling_energy_balance_func,
            dependents=self.cooling_energy_balance_dependents,
            num_eq_sets=1
        )
        return constraints

    def cooling_energy_balance_func(self):
        usable_share = 0.9
        residual = (
            self.inl[1].m.val_SI * (self.outl[1].h.val_SI - self.inl[1].h.val_SI)
            + self.inl[0].m.val_SI * (
                self.outl[0].h.val_SI
                - self.outl[0].h.val_SI / (1 - self.dissipation_ratio.val_SI)
                + self.inl[0].h.val_SI * (self.dissipation_ratio.val_SI / (1 - self.dissipation_ratio.val_SI))
            ) * usable_share
        )
        return residual

    def cooling_energy_balance_dependents(self):
        return [
            self.inl[0].m, self.inl[1].m,
            self.inl[0].h, self.inl[1].h,
            self.outl[0].h, self.outl[1].h
        ]


nw = Network()
nw.units.set_defaults(
    temperature="°C",
    pressure="bar"
)


gas_inlet = Source("gas inlet")
gas_outlet = Sink("gas outlet")

water_inlet = Source("water cold")
water_outlet = Sink("water hot")

compressor = PolynomialCompressorWithCooling("compressor")

c1 = Connection(gas_inlet, "out1", compressor, "in1", label="c1")
c2 = Connection(compressor, "out1", gas_outlet, "in1", label="c2")

b1 = Connection(water_inlet, "out1", compressor, "in2", label="b1")
b2 = Connection(compressor, "out2", water_outlet, "in1", label="b2")


nw.add_conns(c1, c2, b1, b2)

In [28]:
c1.set_attr(fluid={"R290": 1}, m=1, T_dew=10, td_dew=10)
c2.set_attr(T_dew=60, td_dew=50)

b1.set_attr(fluid={"water": 1}, m=1, T=15, p=1)
b2.set_attr(T=25, p=1)
compressor.set_attr(dissipation_ratio=0.1)

nw.solve("design")

You have provided too many parameters: 0 required, 1 supplied. Aborting calculation!


TESPyNetworkError: You have provided too many parameters: 0 required, 1 supplied. Aborting calculation!

- we now have one extra equation, that means we can set one less parameter,
  e.g. cooling mass flow

In [29]:
b1.set_attr(m=None)
nw.solve("design")


 iter  | residual   | progress   | massflow   | pressure   | enthalpy   | fluid      | component  
-------+------------+------------+------------+------------+------------+------------+------------
 1     | 5.81e+04   | 13 %       | 1.39e+00   | 0.00e+00   | 0.00e+00   | 0.00e+00   | 0.00e+00   
 2     | 2.17e-08   | 100 %      | 5.19e-13   | 0.00e+00   | 0.00e+00   | 0.00e+00   | 0.00e+00   
 3     | 1.82e-12   | 100 %      | 4.35e-17   | 0.00e+00   | 0.00e+00   | 0.00e+00   | 0.00e+00   
 4     | 0.00e+00   | 100 %      | 0.00e+00   | 0.00e+00   | 0.00e+00   | 0.00e+00   | 0.00e+00   
Total iterations: 4, Calculation time: 0.00 s, Iterations per second: 1478.82


In [30]:
b1.m.val_SI

0.34934083402916394

In [31]:
from tespy.tools.data_containers import ComponentProperties as dc_cp
from tespy.tools.helpers import TESPyComponentError


class PolynomialCompressorWithCooling(PolynomialCompressor):

    def _preprocess(self, row_idx):
        if not self.eta_recovery.is_set:
            msg = (
                f"The component {self.label} of type {self.__class__.__name__}"
                "requires you to specify the share of heat recovery "
                "eta_recovery."
            )
            raise TESPyComponentError(msg)

        return super()._preprocess(row_idx)

    @staticmethod
    def inlets():
        return ['in1', 'in2']

    @staticmethod
    def outlets():
        return ['out1', 'out2']

    def get_mandatory_constraints(self) -> dict:
        constraints = super().get_mandatory_constraints()
        # this is a dictionary
        constraints["cooling_energy_balance_constraints"] = dc_cmc(
            func=self.cooling_energy_balance_func,
            dependents=self.cooling_energy_balance_dependents,
            num_eq_sets=1
        )
        return constraints

    def get_parameters(self):
        params = super().get_parameters()
        params["eta_recovery"] = dc_cp()
        return params

    def cooling_energy_balance_func(self):
        usable_share = self.eta_recovery.val_SI
        residual = (
            self.inl[1].m.val_SI * (self.outl[1].h.val_SI - self.inl[1].h.val_SI)
            + self.inl[0].m.val_SI * (
                self.outl[0].h.val_SI
                - self.outl[0].h.val_SI / (1 - self.dissipation_ratio.val_SI)
                + self.inl[0].h.val_SI * (self.dissipation_ratio.val_SI / (1 - self.dissipation_ratio.val_SI))
            ) * usable_share
        )
        return residual

    def cooling_energy_balance_dependents(self):
        return [
            self.inl[0].m, self.inl[1].m,
            self.inl[0].h, self.inl[1].h,
            self.outl[0].h, self.outl[1].h
        ]


nw = Network()
nw.units.set_defaults(
    temperature="°C",
    pressure="bar"
)


gas_inlet = Source("gas inlet")
gas_outlet = Sink("gas outlet")

water_inlet = Source("water cold")
water_outlet = Sink("water hot")

compressor = PolynomialCompressorWithCooling("compressor")

c1 = Connection(gas_inlet, "out1", compressor, "in1", label="c1")
c2 = Connection(compressor, "out1", gas_outlet, "in1", label="c2")

b1 = Connection(water_inlet, "out1", compressor, "in2", label="b1")
b2 = Connection(compressor, "out2", water_outlet, "in1", label="b2")


nw.add_conns(c1, c2, b1, b2)

In [32]:
c1.set_attr(fluid={"R290": 1}, m=1, T_dew=10, td_dew=10)
c2.set_attr(T_dew=60, td_dew=25)

b1.set_attr(fluid={"water": 1}, T=15, p=1)
b2.set_attr(T=25, p=1)
compressor.set_attr(dissipation_ratio=0.1, eta_recovery=0.9)

nw.solve("design")


 iter  | residual   | progress   | massflow   | pressure   | enthalpy   | fluid      | component  
-------+------------+------------+------------+------------+------------+------------+------------
 1     | 6.40e+04   | 13 %       | 1.53e+00   | 0.00e+00   | 0.00e+00   | 0.00e+00   | 0.00e+00   
 2     | 2.39e-08   | 100 %      | 5.72e-13   | 0.00e+00   | 0.00e+00   | 0.00e+00   | 0.00e+00   
 3     | 1.82e-12   | 100 %      | 4.35e-17   | 0.00e+00   | 0.00e+00   | 0.00e+00   | 0.00e+00   
 4     | 0.00e+00   | 100 %      | 0.00e+00   | 0.00e+00   | 0.00e+00   | 0.00e+00   | 0.00e+00   
Total iterations: 4, Calculation time: 0.00 s, Iterations per second: 1407.72


In [33]:
compressor.Q_diss.val

-9676.90346751688

In [34]:
b1.m.val_SI * (b2.h.val_SI - b1.h.val_SI)

8709.213120765153

In [35]:
b1.set_attr(m=0.005)
b2.set_attr(T=None)
nw.solve("design")


 iter  | residual   | progress   | massflow   | pressure   | enthalpy   | fluid      | component  
-------+------------+------------+------------+------------+------------+------------+------------
 1     | 8.50e+03   | 23 %       | 0.00e+00   | 0.00e+00   | 1.70e+06   | 0.00e+00   | 0.00e+00   
 2     | 2.15e-04   | 100 %      | 0.00e+00   | 0.00e+00   | 4.29e-02   | 0.00e+00   | 0.00e+00   
 3     | 7.28e-12   | 100 %      | 0.00e+00   | 0.00e+00   | 1.46e-09   | 0.00e+00   | 0.00e+00   
 4     | 1.82e-12   | 100 %      | 0.00e+00   | 0.00e+00   | 3.64e-10   | 0.00e+00   | 0.00e+00   
Total iterations: 4, Calculation time: 0.00 s, Iterations per second: 1196.75


In [36]:
b2.T.val, c2.T.val

(99.60592889710517, 85.00000000002717)

In [37]:
h_2 = c1.h.val_SI + (c2.h.val_SI - c1.h.val_SI) / (1 - compressor.dissipation_ratio.val_SI)
c2.p.val_SI

2116752.919654245

In [38]:
from tespy.tools.fluid_properties import T_mix_ph


T_mix_ph(c2.p.val_SI, h_2, c2.fluid_data) - 273.15

89.04451026537305

In [39]:
compressor.eta_s.val

0.6108450059552428

In [40]:
class PolynomialCompressorWithCooling(PolynomialCompressor):

    def _preprocess(self, row_idx):
        if not self.eta_recovery.is_set:
            msg = (
                f"The component {self.label} of type {self.__class__.__name__}"
                "requires you to specify the share of heat recovery "
                "eta_recovery."
            )
            raise TESPyComponentError(msg)

        return super()._preprocess(row_idx)

    @staticmethod
    def inlets():
        return ['in1', 'in2']

    @staticmethod
    def outlets():
        return ['out1', 'out2']

    def get_mandatory_constraints(self) -> dict:
        constraints = super().get_mandatory_constraints()
        # this is a dictionary
        constraints["cooling_energy_balance_constraints"] = dc_cmc(
            func=self.cooling_energy_balance_func,
            dependents=self.cooling_energy_balance_dependents,
            num_eq_sets=1
        )
        return constraints

    def get_parameters(self):
        params = super().get_parameters()
        params["eta_recovery"] = dc_cp()
        params["td_minimal"] = dc_cp(
            min_val=0
        )
        return params

    def cooling_energy_balance_func(self):
        usable_share = self.eta_recovery.val_SI
        residual = (
            self.inl[1].m.val_SI * (self.outl[1].h.val_SI - self.inl[1].h.val_SI)
            + self.inl[0].m.val_SI * (
                self.outl[0].h.val_SI
                - self.outl[0].h.val_SI / (1 - self.dissipation_ratio.val_SI)
                + self.inl[0].h.val_SI * (self.dissipation_ratio.val_SI / (1 - self.dissipation_ratio.val_SI))
            ) * usable_share
        )
        return residual

    def cooling_energy_balance_dependents(self):
        return [
            self.inl[0].m, self.inl[1].m,
            self.inl[0].h, self.inl[1].h,
            self.outl[0].h, self.outl[1].h
        ]

    def calc_parameters(self):
        super().calc_parameters()

        i = self.inl[0]
        o = self.outl[0]
        h_2 = (
            (o.h.val_SI - i.h.val_SI * self.dissipation_ratio.val_SI)
            / (1 - self.dissipation_ratio.val_SI)
        )

        T_max_compressor_internal = T_mix_ph(
            self.outl[0].p.val_SI,
            h_2,
            self.outl[0].fluid_data,
            self.outl[0].mixing_rule,
            T0=self.outl[0].T.val_SI
        )
        self.td_minimal.val_SI = (
            T_max_compressor_internal
            - self.outl[1].T.val_SI
        )


nw = Network()
nw.units.set_defaults(
    temperature="°C",
    pressure="bar"
)


gas_inlet = Source("gas inlet")
gas_outlet = Sink("gas outlet")

water_inlet = Source("water cold")
water_outlet = Sink("water hot")

compressor = PolynomialCompressorWithCooling("compressor")

c1 = Connection(gas_inlet, "out1", compressor, "in1", label="c1")
c2 = Connection(compressor, "out1", gas_outlet, "in1", label="c2")

b1 = Connection(water_inlet, "out1", compressor, "in2", label="b1")
b2 = Connection(compressor, "out2", water_outlet, "in1", label="b2")


nw.add_conns(c1, c2, b1, b2)

In [41]:
c1.set_attr(fluid={"R290": 1}, m=1, T_dew=10, td_dew=10)
c2.set_attr(T_dew=60, td_dew=25)

b1.set_attr(fluid={"water": 1}, T=15, m=0.05, p=1)
b2.set_attr(p=1)
compressor.set_attr(dissipation_ratio=0.1, eta_recovery=0.9)

nw.solve("design")


 iter  | residual   | progress   | massflow   | pressure   | enthalpy   | fluid      | component  
-------+------------+------------+------------+------------+------------+------------+------------
 1     | 3.81e+04   | 15 %       | 0.00e+00   | 0.00e+00   | 7.63e+05   | 0.00e+00   | 0.00e+00   
 2     | 3.74e-03   | 93 %       | 0.00e+00   | 0.00e+00   | 7.48e-02   | 0.00e+00   | 0.00e+00   
 3     | 4.37e-11   | 100 %      | 0.00e+00   | 0.00e+00   | 8.73e-10   | 0.00e+00   | 0.00e+00   
 4     | 0.00e+00   | 100 %      | 0.00e+00   | 0.00e+00   | 0.00e+00   | 0.00e+00   | 0.00e+00   
Total iterations: 4, Calculation time: 0.00 s, Iterations per second: 1002.46


In [42]:
class PolynomialCompressorWithCooling(PolynomialCompressor):

    def _preprocess(self, row_idx):
        if not self.eta_recovery.is_set:
            msg = (
                f"The component {self.label} of type {self.__class__.__name__}"
                "requires you to specify the share of heat recovery "
                "eta_recovery."
            )
            raise TESPyComponentError(msg)

        return super()._preprocess(row_idx)

    @staticmethod
    def inlets():
        return ['in1', 'in2']

    @staticmethod
    def outlets():
        return ['out1', 'out2']

    def get_mandatory_constraints(self) -> dict:
        constraints = super().get_mandatory_constraints()
        # this is a dictionary
        constraints["cooling_energy_balance_constraints"] = dc_cmc(
            func=self.cooling_energy_balance_func,
            dependents=self.cooling_energy_balance_dependents,
            num_eq_sets=1
        )
        return constraints

    def get_parameters(self):
        params = super().get_parameters()
        params["eta_recovery"] = dc_cp(
            quantity="efficiency"
        )
        params["td_minimal"] = dc_cp(
            min_val=0,
            quantity="temperature_difference"
        )
        params["dp_cooling"] = dc_cp(
            min_val=0,
            structure_matrix=self.dp_structure_matrix,
            func_params={"inconn": 1, "outconn": 1, "dp": "dp_cooling"},
            quantity="pressure"
        )
        return params

    def cooling_energy_balance_func(self):
        residual = (
            self.inl[1].m.val_SI * (self.outl[1].h.val_SI - self.inl[1].h.val_SI)
            + self.inl[0].m.val_SI * (
                self.outl[0].h.val_SI
                - self.outl[0].h.val_SI / (1 - self.dissipation_ratio.val_SI)
                + self.inl[0].h.val_SI * (
                    self.dissipation_ratio.val_SI / (1 - self.dissipation_ratio.val_SI)
                )
            ) * self.eta_recovery.val_SI
        )
        return residual

    def cooling_energy_balance_dependents(self):
        return [
            self.inl[0].m, self.inl[1].m,
            self.inl[0].h, self.inl[1].h,
            self.outl[0].h, self.outl[1].h
        ]

    def calc_parameters(self):
        super().calc_parameters()

        i = self.inl[0]
        o = self.outl[0]
        h_2 = (
            (o.h.val_SI - i.h.val_SI * self.dissipation_ratio.val_SI)
            / (1 - self.dissipation_ratio.val_SI)
        )
        T_max_compressor_internal = T_mix_ph(
            self.outl[0].p.val_SI,
            h_2,
            self.outl[0].fluid_data,
            self.outl[0].mixing_rule,
            T0=self.outl[0].T.val_SI
        )
        self.td_minimal.val_SI = (
            T_max_compressor_internal
            - self.outl[1].T.val_SI
        )

        self.dp_cooling.val_SI = self.inl[1].p.val_SI - self.outl[1].p.val_SI


nw = Network()
nw.units.set_defaults(
    temperature="°C",
    pressure="bar"
)


gas_inlet = Source("gas inlet")
gas_outlet = Sink("gas outlet")

water_inlet = Source("water cold")
water_outlet = Sink("water hot")

compressor = PolynomialCompressorWithCooling("compressor")

c1 = Connection(gas_inlet, "out1", compressor, "in1", label="c1")
c2 = Connection(compressor, "out1", gas_outlet, "in1", label="c2")

b1 = Connection(water_inlet, "out1", compressor, "in2", label="b1")
b2 = Connection(compressor, "out2", water_outlet, "in1", label="b2")


nw.add_conns(c1, c2, b1, b2)

In [43]:
c1.set_attr(fluid={"R290": 1}, m=1, T_dew=10, td_dew=10)
c2.set_attr(T_dew=60, td_dew=25)

b1.set_attr(fluid={"water": 1}, T=15, m=0.05, p=1)
b2.set_attr(p=0.9)
compressor.set_attr(dissipation_ratio=0.1, eta_recovery=0.9)

nw.solve("design")


 iter  | residual   | progress   | massflow   | pressure   | enthalpy   | fluid      | component  
-------+------------+------------+------------+------------+------------+------------+------------
 1     | 3.81e+04   | 15 %       | 0.00e+00   | 0.00e+00   | 7.63e+05   | 0.00e+00   | 0.00e+00   
 2     | 3.74e-03   | 93 %       | 0.00e+00   | 0.00e+00   | 7.48e-02   | 0.00e+00   | 0.00e+00   
 3     | 4.37e-11   | 100 %      | 0.00e+00   | 0.00e+00   | 8.73e-10   | 0.00e+00   | 0.00e+00   
 4     | 0.00e+00   | 100 %      | 0.00e+00   | 0.00e+00   | 0.00e+00   | 0.00e+00   | 0.00e+00   
Total iterations: 4, Calculation time: 0.00 s, Iterations per second: 1393.46


In [44]:
nw.print_results()


##### RESULTS (PolynomialCompressorWithCooling) #####
+------------+----------+-----------+---------------------+----------------+--------------+
|            |       pr |        dp |   dissipation_ratio |   eta_recovery |   dp_cooling |
|------------+----------+-----------+---------------------+----------------+--------------|
| compressor | 3.33e+00 | -1.48e+01 |            1.00e-01 |       9.00e-01 |     1.00e-01 |
+------------+----------+-----------+---------------------+----------------+--------------+
##### RESULTS (Connection) #####
+----+-----------+-----------+-----------+-----------+-----------+---------+
|    |         m |         p |         h |         T |         x | phase   |
|----+-----------+-----------+-----------+-----------+-----------+---------|
| b1 | 5.000e-02 | 1.000e+00 | 6.308e+04 | 1.500e+01 | 0.000e+00 | l       |
| b2 | 5.000e-02 | 9.000e-01 | 2.373e+05 | 5.666e+01 | 0.000e+00 | l       |
| c1 | 1.000e+00 | 6.366e+00 | 6.040e+05 | 2.000e+01 | 1.000e+00 | 